# CSP - N Queen - Genitic Algorithm

In [1]:
import random

def random_chromosome(size): #making random chromosomes 
    return [ random.randint(1, nq) for _ in range(nq) ]

def fitness(chromosome):
    horizontal_collisions = sum([chromosome.count(queen)-1 for queen in chromosome])/2
    diagonal_collisions = 0

    n = len(chromosome)
    left_diagonal = [0] * 2*n
    right_diagonal = [0] * 2*n
    for i in range(n):
        left_diagonal[i + chromosome[i] - 1] += 1
        right_diagonal[len(chromosome) - i + chromosome[i] - 2] += 1

    diagonal_collisions = 0
    for i in range(2*n-1):
        counter = 0
        if left_diagonal[i] > 1:
            counter += left_diagonal[i]-1
        if right_diagonal[i] > 1:
            counter += right_diagonal[i]-1
        diagonal_collisions += counter / (n-abs(i-n+1))
    
    return int(maxFitness - (horizontal_collisions + diagonal_collisions)) #28-(2+3)=23

def probability(chromosome, fitness):
    return fitness(chromosome) / maxFitness

def random_pick(population, probabilities):
    populationWithProbabilty = zip(population, probabilities)
    total = sum(w for c, w in populationWithProbabilty)
    r = random.uniform(0, total)
    upto = 0
    for c, w in zip(population, probabilities):
        if upto + w >= r:
            return c
        upto += w
    assert False, "Shouldn't get here"
        
def reproduce(x, y): #doing cross_over between two chromosomes
    n = len(x)
    c = random.randint(0, n - 1)
    return x[0:c] + y[c:n]

def mutate(x):  #randomly changing the value of a random index of a chromosome
    n = len(x)
    c = random.randint(0, n - 1)
    m = random.randint(1, n)
    x[c] = m
    return x

def genetic_queen(population, fitness):
    mutation_probability = 0.03
    new_population = []
    probabilities = [probability(n, fitness) for n in population]
    for i in range(len(population)):
        x = random_pick(population, probabilities) #best chromosome 1
        y = random_pick(population, probabilities) #best chromosome 2
        child = reproduce(x, y) #creating two new chromosomes from the best 2 chromosomes
        if random.random() < mutation_probability:
            child = mutate(child)
        print_chromosome(child)
        new_population.append(child)
        if fitness(child) == maxFitness: break
    return new_population

def print_chromosome(chrom):
    print("Chromosome = {},  Fitness = {}"
        .format(str(chrom), fitness(chrom)))

if __name__ == "__main__":
    nq = int(input("Enter Number of Queens: ")) #say N = 8
    maxFitness = (nq*(nq-1))/2  # 8*7/2 = 28
    population = [random_chromosome(nq) for _ in range(100)]
    
    generation = 1

    while not maxFitness in [fitness(chrom) for chrom in population]:
        print("=== Generation {} ===".format(generation))
        population = genetic_queen(population, fitness)
        print("")
        print("Maximum Fitness = {}".format(max([fitness(n) for n in population])))
        generation += 1
    chrom_out = []
    print("Solved in Generation {}!".format(generation-1))
    for chrom in population:
        if fitness(chrom) == maxFitness:
            print("");
            print("One of the solutions: ")
            chrom_out = chrom
            print_chromosome(chrom)
            
    board = []

    for x in range(nq):
        board.append(["x"] * nq)
        
    for i in range(nq):
        board[nq-chrom_out[i]][i]="Q"
            

    def print_board(board):
        for row in board:
            print (" ".join(row))
            
    print()
    print_board(board)
            
           
            
    


Enter Number of Queens: 4
=== Generation 1 ===
Chromosome = [3, 1, 4, 4],  Fitness = 5
Chromosome = [2, 4, 2, 1],  Fitness = 4
Chromosome = [2, 1, 4, 4],  Fitness = 4
Chromosome = [1, 2, 4, 4],  Fitness = 4
Chromosome = [2, 2, 3, 1],  Fitness = 4
Chromosome = [3, 3, 2, 3],  Fitness = 2
Chromosome = [3, 4, 4, 4],  Fitness = 2
Chromosome = [4, 1, 3, 3],  Fitness = 4
Chromosome = [4, 1, 1, 1],  Fitness = 2
Chromosome = [3, 2, 3, 4],  Fitness = 4
Chromosome = [4, 3, 2, 3],  Fitness = 4
Chromosome = [4, 1, 2, 2],  Fitness = 4
Chromosome = [4, 1, 3, 2],  Fitness = 5
Chromosome = [4, 2, 1, 2],  Fitness = 4
Chromosome = [4, 2, 1, 2],  Fitness = 4
Chromosome = [4, 2, 1, 4],  Fitness = 4
Chromosome = [3, 1, 4, 1],  Fitness = 5
Chromosome = [2, 4, 2, 4],  Fitness = 4
Chromosome = [2, 2, 1, 4],  Fitness = 4
Chromosome = [4, 4, 1, 4],  Fitness = 3
Chromosome = [2, 2, 3, 1],  Fitness = 4
Chromosome = [1, 3, 3, 3],  Fitness = 2
Chromosome = [2, 2, 3, 1],  Fitness = 4
Chromosome = [2, 2, 1, 1],  Fitne

# CSP - N Queen - Hill Climbing

In [6]:
import math
import sys
import random
import numpy as np
import matplotlib.pyplot as plt

MAXQ = 100
MAX_ITER = 1000


def in_conflict(column, row, other_column, other_row):
    """
    Checks if two locations are in conflict with each other.
    :param column: Column of queen 1.
    :param row: Row of queen 1.
    :param other_column: Column of queen 2.
    :param other_row: Row of queen 2.
    :return: True if the queens are in conflict, else False.
    """
    if column == other_column:
        return True  # Same column
    if row == other_row:
        return True  # Same row
    if abs(column - other_column) == abs(row - other_row):
        return True  # Diagonal

    return False


def in_conflict_with_another_queen(row, column, board):
    """
    Checks if the given row and column correspond to a queen that is in conflict with another queen.
    :param row: Row of the queen to be checked.
    :param column: Column of the queen to be checked.
    :param board: Board with all the queens.
    :return: True if the queen is in conflict, else False.
    """
    for other_column, other_row in enumerate(board):
        if in_conflict(column, row, other_column, other_row):
            if row != other_row or column != other_column:
                return True
    return False


def count_conflicts(board):
    """
    Counts the number of queens in conflict with each other.
    :param board: The board with all the queens on it.
    :return: The number of conflicts.
    """
    cnt = 0

    for queen in range(0, len(board)):
        for other_queen in range(queen + 1, len(board)):
            if in_conflict(queen, board[queen], other_queen, board[other_queen]):
                cnt += 1

    return cnt


def evaluate_state(board):
    """
    Evaluation function. The maximal number of queens in conflict can be:
    1 + 2 + 3 + 4 + ... + (nqueens-1) = ((nqueens-1) * nqueens) / 2
    Since we want to do ascending local searches, the evaluation function returns
    (((nqueens-1) * nqueens) / 2) - countConflicts().
    :param board: list/array representation of columns and the row of the queen on that column
    :return: evaluation score
    """
    return (len(board) - 1) * len(board) / 2 - count_conflicts(board)


def print_board(board):
    """
    Prints the board in a human-readable format in the terminal.
    :param board: The board with all the queens.
    """
    for row in range(len(board)):
        line = ''
        for column in range(len(board)):
            if board[column] == row:
                line += 'Q' if in_conflict_with_another_queen(row, column, board) else 'q'
            else:
                line += '.'
        print(line)
    print("")


def init_board(nqueens):
    """
    :param nqueens integer for the number of queens on the board
    :returns list/array representation of columns and the row of the queen on that column
    """
    board = []

    for column in range(nqueens):
        board.append(random.randint(0, nqueens - 1))

    return board


def Hill_Climbing(board):
    """
    Hill Climbing algorithm in Readme file.
    :param board: list/array representation of columns and the row of the queen on that column
    """
    i = 0
    optimum = (len(board) - 1) * len(board) / 2
    evaluation = [evaluate_state(board)]

    while evaluate_state(board) != optimum:
        i += 1
        print(f"Iteration {i}: Evaluation = {evaluate_state(board)}")

        if i == MAX_ITER:  # Give up after MAX_ITER tries.
            break

        max_score_of_each_column = []
        row_resulting_in_max_score = []

        for col in range(len(board)):
            col_scores = []

            for row in range(len(board)):
                new_board = board.copy()
                new_board[col] = row
                col_scores.append(evaluate_state(new_board))

            if max(col_scores) > evaluate_state(board):
                max_score_of_each_column.append(max(col_scores))
                row_resulting_in_max_score.append(np.argmax(col_scores))
            else:
                max_score_of_each_column.append(False)
                row_resulting_in_max_score.append(False)

        if max(max_score_of_each_column):
            maximizing_col = np.argmax(max_score_of_each_column)
            maximizing_row = row_resulting_in_max_score[maximizing_col]
            board[maximizing_col] = maximizing_row

        evaluation.append(evaluate_state(board))

    if evaluate_state(board) == optimum:
        print('\nSolved Puzzle!')

    print('\nFinal State:')
    print_board(board)

def main():
    n_queens = 4
    board = init_board(n_queens)
    print('\nInitial Board:')
    print_board(board)
    Hill_Climbing(board)


# This line is the starting point of the program.
if __name__ == "__main__":
    main()


Initial Board:
..q.
....
...q
QQ..

Iteration 1: Evaluation = 5.0

Solved Puzzle!

Final State:
..q.
q...
...q
.q..

